In [1]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])

In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [5]:
# Normalize the text data
df = normalize_text(df)

In [6]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

C:\Users\karti\AppData\Local\Temp\ipykernel_16588\1508851655.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [14]:
vectorizer = CountVectorizer()

In [15]:
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}


In [7]:
mlflow.set_tracking_uri("https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow")
dagshub.init(repo_owner='Iamkartikey44', repo_name='mlops-mini-project', mlflow=True)
# Set the experiment name
mlflow.set_experiment("LOR Hyperparameter Tuning")

Accessing as Iamkartikey44

Initialized MLflow to track repo "Iamkartikey44/mlops-mini-project"

Repository Iamkartikey44/mlops-mini-project initialized!

2025/11/15 00:34:25 INFO mlflow.tracking.fluent: Experiment with name 'LOR Hyperparameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1ecc3433557f4b70bee8ccc16ad52f05', creation_time=1763147060461, experiment_id='4', last_update_time=1763147060461, lifecycle_stage='active', name='LOR Hyperparameter Tuning', tags={}>

In [ ]:
with  mlflow.start_run():

    #Perform grid search
    grid_search = GridSearchCV(LogisticRegression(),param_grid,cv=5,scoring='f1',n_jobs=-1)
    grid_search.fit(X_train,y_train)

    #Log each parameter combination as a child run
    for params,mean_score,std_score in zip(grid_search.cv_results_['params'],grid_search.cv_results_['mean_test_score'],grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name='LR with params: {params}',nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train,y_train)

            #Model Evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test,y_pred)
            precision = precision_score(y_test,y_pred)
            recall = recall_score(y_test,y_pred)
            f1 =f1_score(y_test,y_pred)

            #Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("Mean_CV_Score",mean_score)
            mlflow.log_metric("STD_CV_Score",std_score)
            mlflow.log_metric("Accuracy",accuracy)
            mlflow.log_metric('Precision',precision)
            mlflow.log_metric("Recall",recall)
            mlflow.log_metric("F1_score",f1)

            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")
            

    #Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score",best_score)

    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    #Save and Log the notebook
    notebook_path = "exp1_bow_vs_tfidf.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)


    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")


            

Mean CV Score: 0.7051810430308207, Std CV Score: 0.014168003094990942
Accuracy: 0.7397590361445783
Precision: 0.7752027809965237
Recall: 0.6591133004926109
F1 Score: 0.7124600638977636


2025/11/15 00:50:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {params} at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4/runs/327c03b0e927468d9a0d0bd230f1b10e.
2025/11/15 00:50:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4.


Mean CV Score: 0.7792109402770179, Std CV Score: 0.010740092596030877
Accuracy: 0.7893975903614457
Precision: 0.7805825242718447
Recall: 0.7921182266009852
F1 Score: 0.7863080684596577


2025/11/15 00:50:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {params} at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4/runs/c5199e4d8ea94339aeff8a2111a8cfe3.
2025/11/15 00:50:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4.


Mean CV Score: 0.7829215927546672, Std CV Score: 0.008058669641280986
Accuracy: 0.7826506024096386
Precision: 0.77431906614786
Recall: 0.7842364532019704
F1 Score: 0.7792462065589819


...

Mean CV Score: 0.7710309254394809, Std CV Score: 0.010849680172073033
Accuracy: 0.7831325301204819
Precision: 0.7816550348953141
Recall: 0.7724137931034483
F1 Score: 0.7770069375619425


2025/11/15 00:51:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {params} at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4/runs/d51d6260655247d3ae0d8e601336d59d.
2025/11/15 00:51:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4.


Mean CV Score: 0.779151984138562, Std CV Score: 0.00473765823574164
Accuracy: 0.7807228915662651
Precision: 0.7702702702702703
Recall: 0.7862068965517242
F1 Score: 0.7781569965870307


2025/11/15 00:51:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {params} at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4/runs/a4c6b577011c417786dcae8113ae7c68.
2025/11/15 00:51:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Iamkartikey44/mlops-mini-project.mlflow/#/experiments/4.


Best Params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 Score: 0.7904116981776645
